We're going to use the dataset on [Kaggle](https://www.kaggle.com/datasets/federalreserve/interest-rates). It contains interest rate data since 1954. The federal funds rate is the interest rate at which banks trade federal funds overnight. The Fed sets a **target** for this, but due to the free market, the **effective** funds rate will be different. Since we're predicting the Fed's decision, we're going to predict the target rate.

According to the dataset description:
> The unemployment rate represents the number of unemployed as a seasonally adjusted percentage of the labor force. The inflation rate reflects the monthly change in the Consumer Price Index of products excluding food and energy.

In [1]:
import pandas as pd
df = pd.read_csv('index.csv')

First, let's see how many missing values we have.

In [2]:
df.isnull().sum()

Year                              0
Month                             0
Day                               0
Federal Funds Target Rate       442
Federal Funds Upper Target      801
Federal Funds Lower Target      801
Effective Federal Funds Rate    152
Real GDP (Percent Change)       654
Unemployment Rate               152
Inflation Rate                  194
dtype: int64

It looks like there are a lot of missing values in `Federal Funds Upper Target` and `Federal Funds Lower Target`. Since this range system was only adopted more recently, we can simply convert them back to single values by calculating their average. Also, there are way too many missing values in `Real GDP (Percent Change)` so we're just going to drop it.

In [3]:
df['Federal Funds Rate'] = (df['Federal Funds Upper Target'] + df['Federal Funds Lower Target']) / 2
df = df.drop(['Federal Funds Upper Target', 'Federal Funds Lower Target', 'Real GDP (Percent Change)'], axis=1)
print(df.head())

   Year  Month  Day  Federal Funds Target Rate  Effective Federal Funds Rate  \
0  1954      7    1                        NaN                          0.80   
1  1954      8    1                        NaN                          1.22   
2  1954      9    1                        NaN                          1.06   
3  1954     10    1                        NaN                          0.85   
4  1954     11    1                        NaN                          0.83   

   Unemployment Rate  Inflation Rate  Federal Funds Rate  
0                5.8             NaN                 NaN  
1                6.0             NaN                 NaN  
2                6.1             NaN                 NaN  
3                5.7             NaN                 NaN  
4                5.3             NaN                 NaN  
